In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection as ms
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
from imblearn.over_sampling import SMOTE

In [5]:
#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data3_1/'+'Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

#스케일링 먼저 하고 ndarray를 dataframe으로 변환
scaler = StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

In [6]:
#한 사람당 데이터 수
Count_1=int(181*0.1)
#한 사람당 데이터 수
Count_2=181

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

for i in range(10):
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
    
    #SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train,y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    
    #모델 정의, 예측
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    
    predict=model.predict(X_test)
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))
    #f1score
    f1 = f1_score(y_test,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    print()
    
    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=list[2]
    Result[i][2]=list[0]
    Result[i][3]=list[1]
    print("{} : {}".format(list[2],f1))

[0]Accuracy : 0.49246649996794256
[0]F1score : 0.4746650929831057
[0]Precision : 0.48136517125643424
[0]Recall : 0.49246649996794256

0.4746650929831057 : 0.4746650929831057
[1]Accuracy : 0.8875961476310517
[1]F1score : 0.8858922043001466
[1]Precision : 0.8882576977314791
[1]Recall : 0.8875961476310517

0.8858922043001466 : 0.8858922043001466
[2]Accuracy : 0.877983423692902
[2]F1score : 0.8758211174879598
[2]Precision : 0.8792596078589864
[2]Recall : 0.877983423692902

0.8758211174879598 : 0.8758211174879598
[3]Accuracy : 0.8455691509084168
[3]F1score : 0.8418507806433724
[3]Precision : 0.8457766201771114
[3]Recall : 0.8455691509084168

0.8418507806433724 : 0.8418507806433724
[4]Accuracy : 0.863647475858697
[4]F1score : 0.8609920051436837
[4]Precision : 0.8640622990536646
[4]Recall : 0.863647475858697

0.8609920051436837 : 0.8609920051436837
[5]Accuracy : 0.7659899970579582
[5]F1score : 0.7543077005923964
[5]Precision : 0.7698894977740405
[5]Recall : 0.7659899970579582

0.7543077005923

In [7]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.492466,0.474665,0.481365,0.492466
1,0.887596,0.885892,0.888258,0.887596
2,0.877983,0.875821,0.879260,0.877983
3,0.845569,0.841851,0.845777,0.845569
4,0.863647,0.860992,0.864062,0.863647
5,0.765990,0.754308,0.769889,0.765990
6,0.835935,0.832187,0.838493,0.835935
7,0.828462,0.823907,0.827127,0.828462
8,0.843242,0.839552,0.842840,0.843242
9,0.845689,0.841611,0.848012,0.845689


In [8]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.8086579944659528
Average of F1-Score 0.8030784586587589
Average of Precision 0.8085082639959957
Average of Recall 0.8086579944659528


In [9]:
Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix2.xlsx')